In [ ]:

h = rand(21,202)
J = rand(21,202,21,202)

@time res = run_evolution(Int8.(rand(1:21,(,1000))), h, J, p = 0, N_steps = 100, N_points = 10);

## tests

In [ ]:
using Revise, Genie, PyPlot, Statistics, Random, JLD2, Distributions, StatsBase, DCAUtils, DelimitedFiles, BenchmarkTools, LinearAlgebra, PyPlot
import KitMSA: fasta2matrix, letter2num, num2letter, extract_params, read_par_BM, set_max_field_to_0, symmetrize_J

h, J = extract_params("../scra_data_beta/Parameters_conv_Matteo_pc.dat"; q = 21);
J = permutedims(J, [1,3,2,4]);

L = 202; q = 21; N_chains = 2; rng = random_gens(N_chains); temp = 1.; no_cod_bias = 
Dict(x => (1/length(amino2cod[cod2amino[x]])) for x in keys(cod2amino)); codon_usage = no_cod_bias;
chains = [Chain(Int8.(rand(1:q,L)), q, rng[n]) for n in 1:N_chains]; codon_net = Genie.create_nested_codon_dict(); length_of_moves = create_length_dict(codon_net);
seq_site = rand(chains[1].generator, findall(x -> x != 21, chains[1].seq)); nucleo_site = rand(chains[1].generator,1:3);

length_of_moves[chains[1].DNA[seq_site], nucleo_site]
codon_list = accessible_codons(chains[1].DNA[seq_site], codon_net, nucleo_site)
log_prob = zeros(size(codon_list,1))

@code_warntype Genie.accessible_codons_old(chains[1].DNA[seq_site], codon_net, nucleo_site)
@code_warntype Genie.accessible_codons(chains[1].DNA[seq_site], codon_net, nucleo_site)

@btime log_prob!($(log_prob), $(chains[1]), $(h), $(J), $(codon_usage), $(codon_list), $(temp), $(seq_site), $(L))
@code_warntype log_prob!(log_prob, chains[1], h, J, codon_usage, codon_list, temp, seq_site, L)

@btime prob_cond!($(chains[1]), $(h), $(J), $(seq_site), $(nucleo_site), $(codon_net), $(codon_usage), $(length_of_moves),$(temp), $(L))
@code_warntype prob_cond!(chains[1], h, J, seq_site, nucleo_site, codon_net, codon_usage, length_of_moves, temp, L)


#@btime prob_cond2!($(chains[1]), $(h), $(J), $(seq_site), $(nucleo_site), $(codon_net), $(codon_usage), $(temp), $(L))


In [ ]:
## test proposal of metropolis

target = 63/64
n_it = 10^4
all_codons = vcat([amino2cod[i] for i in 1:20]...)
for cod in all_codons
    res = []
    for i in 1:n_it
        push!(res,Genie.del_ins_codon_sampling(Genie.get_accessible_nucleo_for_del_ins(cod)[2]))
    end
    
    println("Old method: Codon $(cod) difference from target $(sum(res .== cod)/n_it  - target)")
end


target = 63/64
n_it = 10^4
all_codons = vcat([amino2cod[i] for i in 1:20]...)
for cod in all_codons
    res = []
    for i in 1:n_it
        push!(res, Genie.proposed_codon(cod, all_codons))
    end
    
    println("New method: Codon $(cod) difference from target $(sum(res .== nothing)/n_it - target)")
end

target = 1/64
n_it = 10^5
all_codons = vcat([amino2cod[i] for i in 1:20]...)
res = [Genie.del_ins_codon_sampling(Genie.get_accessible_nucleo_for_del_ins("---")[2]) for _ in 1:n_it]

for cod in all_codons
    println("Old method: Codon $(cod) difference from target $(sum(res .== cod)/n_it  - target)")
end


target = 1/64
n_it = 10^5
all_codons = vcat([amino2cod[i] for i in 1:20]...)
res = [Genie.proposed_codon(cod, all_codons) for _ in 1:n_it]

for cod in all_codons
    println("New method: Codon $(cod) difference from target $(sum(res .== cod)/n_it  - target)")
end
    



    
    


## short term test

here i test against two experiments, for tem the correlation should be 0.78, for pse around 0.86
the final hamming for tem should be 17/18, for pse 23/24


In [ ]:
using Revise, Genie, PyPlot, Statistics, Random, JLD2, Distributions, StatsBase, DCAUtils, DelimitedFiles, BenchmarkTools, LinearAlgebra, PyPlot
import KitMSA: fasta2matrix, letter2num, num2letter, extract_params, read_par_BM, set_max_field_to_0, symmetrize_J

h, J = extract_params("../scra_data_beta/Parameters_conv_Matteo_pc.dat"; q = 21);
J = permutedims(J, [1,3,2,4]);

path_wt_tem = "../Gen.jl/data/alignments/refs/TEM1_pfam_DNA.fasta"; path_wt_pse = 
"../Gen.jl/data/alignments/refs/PSE1_pfam_DNA.fasta"; wt_tem = join(readdlm(path_wt_tem, 
        skipstart = 1)); wt_pse = join(readdlm(path_wt_pse, skipstart = 1)); L_tem = 
Int64(length(wt_tem)/3); tem_DNA_seq = 
[wt_tem[((i-1)*3 +1):(i*3)] for i in 1:L_tem]; tem_amino_seq = [cod2amino[codon] 
    for codon in tem_DNA_seq]; L_pse = Int64(length(wt_pse)/3); pse_DNA_seq = 
[wt_pse[((i-1)*3 +1):(i*3)] for i in 1:L_pse]; pse_amino_seq = [cod2amino[codon] 
    for codon in pse_DNA_seq];

path_fantini = "../Gen.jl/data/alignments/experimental/fantini.max.6.gaps.gen12.fa"; MSA_tem_exp = 
Int8.(fasta2matrix(path_fantini)); f1_tem,f2_tem = 
DCAUtils.compute_weighted_frequencies(Int8.(MSA_tem_exp'), 22, 0.); conn_tem = f2_tem .- f1_tem' * f1_tem;

MSA_pse_exp = Int8.(fasta2matrix("../scra_data_beta/cleaned_aligned_stiffler_MSA_big.fasta")); f1_pse,f2_pse = 
DCAUtils.compute_weighted_frequencies(Int8.(MSA_pse_exp'), 22, 0.); conn_pse = f2_pse .- f1_pse' * f1_pse;

In [ ]:
##test with fantini tem
start_msa = hcat([tem_DNA_seq for i in 1:size(MSA_tem_exp,1)]...); @time res_tem = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.14, 
    N_steps = 109,  
    N_points = 20);

f1_sil_tem, f2_sil_tem = DCAUtils.compute_weighted_frequencies(Int8.(res_tem.step_msa[end]), 22, 0.); conn_sil_tem = f2_sil_tem .- 
f1_sil_tem' * f1_sil_tem;

println(""); println("Final tem sil hamming : $(mean(ham_dist(tem_amino_seq, Int8.(res_tem.step_msa[end])))) +- $(std(ham_dist(tem_amino_seq, Int8.(res_tem.step_msa[end]))))"); println(
    "Final tem exp hamming : $(mean(ham_dist(tem_amino_seq, Int8.(MSA_tem_exp')))) +- $(std(ham_dist(tem_amino_seq, Int8.(MSA_tem_exp'))))")  

println(""); println("Final tem sil energy : $(mean(energy(Int8.(res_tem.step_msa[end]), h, J))) +- $(std(energy(Int8.(res_tem.step_msa[end]), h, J)))"); println(
    "Final tem exp energy : $(mean(energy(Int8.(MSA_tem_exp'), h, J))) +- $(std(energy(Int8.(MSA_tem_exp'), h, J)))")  

println(""); println("1-point tem correlation $(corspearman(f1_tem[:], f1_sil_tem[:]))")

close("all"); plt.plot(f1_tem[:], f1_sil_tem[:], "o"); plt.xscale("log"); plt.yscale("log"); savefig("../prova_tem.png")


#test with stiffler pse
start_msa = hcat([pse_DNA_seq for i in 1:size(MSA_pse_exp,1)]...); @time res = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.4, 
    N_steps = 120,  
    N_points = 20);

f1_sil_pse, f2_sil_pse = DCAUtils.compute_weighted_frequencies(Int8.(res.step_msa[end]), 
    22, 0.); conn_sil_pse = f2_sil_pse .- f1_sil_pse' * f1_sil_pse;

println(""); println("1-point correlation pse $(corspearman(f1_pse[:], f1_sil_pse[:]))")

println(""); println("Final pse sil hamming : $(mean(ham_dist(pse_amino_seq, Int8.(res.step_msa[end])))) +- $(std(ham_dist(pse_amino_seq, Int8.(res.step_msa[end]))))"); println(
    "Final pse exp hamming : $(mean(ham_dist(pse_amino_seq, Int8.(MSA_pse_exp')))) +- $(std(ham_dist(pse_amino_seq, Int8.(MSA_pse_exp'))))")  

println(""); println("Final pse sil energy : $(mean(energy(Int8.(res.step_msa[end]), h, J))) +- $(std(energy(Int8.(res.step_msa[end]), h, J)))"); println(
    "Final pse exp energy : $(mean(energy(Int8.(MSA_pse_exp'), h, J))) +- $(std(energy(Int8.(MSA_pse_exp'), h, J)))")  


close("all"); plt.plot(f1_pse[:], f1_sil_pse[:], "o"); plt.xscale("log"); plt.yscale("log"); savefig("../prova_pse.png")


## Long term test


here i test the generativity for the dbd family


In [ ]:
@load "../data_Genie/pars_dbd.jld2"

dbd_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/DBD_alignment.uniref90.cov80.a2m", 0.9);

wt_thornton = Int8.(readdlm("../Gen.jl/data/alignments/refs/wt_thornton")[:]);
start_msa = hcat([dbd_msa[:,1] for i in 1:1000]...); @time res = run_evolution(start_msa, 
    h_dbd, 
    J_dbd,
    p = 0.1, 
    temp = 1., 
    N_steps = 600000,  
    N_points = 20, 
    verbose = true);

check_equilibration("../figures/dbd_p0.1/", dbd_msa, res.step_msa, res.steps)

Genie.check_timescales(res.step_msa, 
    dbd_msa, h_dbd, J_dbd, res.steps, "../figures/dbd/dynamics.png");


In [ ]:
h, J = extract_params("../scra_data_beta/Parameters_conv_Matteo_pc.dat"; q = 21);
J = permutedims(J, [1,3,2,4]);



beta_msa  = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 0.9);

start_msa = hcat([pse_DNA_seq for i in 1:1000]...); @time res = run_evolution(start_msa, 
    h, 
    J,
    p = 0.5, 
    temp = 1., 
    N_steps = 10^7,  
    N_points = 20, 
    verbose = true);

check_equilibration("../figures/beta/", beta_msa, res.step_msa, res.steps)

Genie.check_timescales(res.step_msa, 
    beta_msa, h, J, res.steps, "../figures/beta/dynamics.png");


In [ ]:
start_msa = hcat([wt_hinds for i in 1:1000]...); @time res = run_evolution(start_msa, 
           h, 
           J,
           p = 0.5, 
           temp = 1., 
           N_steps = 4000000,  
           N_points = 150, 
           verbose = true); @time check_equilibration("../figures/dhfr/", 
    dhfr_msa, res.step_msa); @time Genie.check_timescales(res.step_msa, 
    dhfr_msa, h, J, res.steps, "../figures/dhfr/dynamics.png");
